In [100]:
from Bio import SeqIO
f = open('dna2.fasta')
sequences = SeqIO.parse(f,'fasta')

### Part 1
How many records are in the file? A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, and the rest of the line is an optional description of the entry. There should be no space between the ">" and the first letter of the identifier. 

What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? Is there more than one longest or shortest sequence? What are their identifiers? 

In [101]:
i = 0
seqlen = []
identifier = []

for fasta in sequences:
    seqlen.append(len(str(fasta.seq)))
    identifier.append(fasta.id)
    i += 1

nseqs = i
minlen = min(seqlen)
maxlen = max(seqlen)

print(seqlen)
print('Minimum length',minlen)
print('Maximum length',maxlen)

minlen_inds = [i for i in range(nseqs) if seqlen[i]==minlen]
minlen_ids = [identifier[minlen_inds[j]] for j in range(len(minlen_inds))]
print("Minimum length sequences:", minlen_ids)

maxlen_inds = [i for i in range(nseqs) if seqlen[i]==maxlen]
maxlen_ids = [identifier[maxlen_inds[j]] for j in range(len(maxlen_inds))]
print("Maximum length sequences:", maxlen_ids)

#for fasta in sequences:
#    if (len(str(fasta.seq)) == minlen):
#        print('Minimum length sequence:', fasta.id)
#    elif (len(str(fasta.seq)) == maxlen):
#        print('Maximum length sequence:', fasta.id)

f.close()

[4635, 1151, 4894, 3511, 4076, 2867, 442, 890, 967, 4338, 1352, 4564, 4804, 964, 2095, 1432, 115, 2646]
Minimum length 115
Maximum length 4894
Minimum length sequences: ['gi|142022655|gb|EQ086233.1|346']
Maximum length sequences: ['gi|142022655|gb|EQ086233.1|255']


In [103]:
for id in identifier:
    print(id)
    
print(len(identifier))

gi|142022655|gb|EQ086233.1|91
gi|142022655|gb|EQ086233.1|304
gi|142022655|gb|EQ086233.1|255
gi|142022655|gb|EQ086233.1|45
gi|142022655|gb|EQ086233.1|396
gi|142022655|gb|EQ086233.1|250
gi|142022655|gb|EQ086233.1|322
gi|142022655|gb|EQ086233.1|88
gi|142022655|gb|EQ086233.1|594
gi|142022655|gb|EQ086233.1|293
gi|142022655|gb|EQ086233.1|75
gi|142022655|gb|EQ086233.1|454
gi|142022655|gb|EQ086233.1|16
gi|142022655|gb|EQ086233.1|584
gi|142022655|gb|EQ086233.1|4
gi|142022655|gb|EQ086233.1|277
gi|142022655|gb|EQ086233.1|346
gi|142022655|gb|EQ086233.1|527
18


### Part 2
Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, and answer the following questions: what is the length of the longest ORF in the file? What is the identifier of the sequence containing the longest ORF? For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence.

In [123]:
with open('dna2.fasta') as f:
    seq_dict = SeqIO.to_dict(SeqIO.parse(f,'fasta'))
# with open automatically closes the file

In [130]:
def orf(seq, start):
    # Look for starts
    orf_starts, orf_stops, orf_lens = [], [], []
    stop_codons = ['TAA', 'TAG', 'TGA']
    for ind in range(start,len(seq),3): 
        if (seq[ind:ind+3]=='ATG'):
            orf_starts.append(ind)
        
    for ind2 in range(start+3,len(seq),3):   
        # for more efficiency can check if there is
        # at least one orf_start and search starting there
        if (seq[ind2:ind2+3] in stop_codons):
            orf_stops.append(ind2)   # start of first nucleotide of last codon
            
    #print("Starts: ", orf_starts)
    #print("Stops: ", orf_stops)
    
    orf_start_pos = []
    for i in range(len(orf_starts)):    
        orf_next_stops = [el for el in orf_stops if el > orf_starts[i]]
        if len(orf_next_stops) > 0:
            orf_lens.append(orf_next_stops[0]+2 - orf_starts[i] + 1)
            orf_start_pos.append(orf_starts[i]+1)   # e.g., index 0 is position 1
            #print(f'Has ORF at ind {orf_starts[i]} through {orf_next_stops[0]+2}')
            #print('Length: ', orf_lens[-1])
            #print(seq[orf_starts[i]:orf_starts[i]+3])   # Should be ATG
            #print(seq[orf_next_stops[0]:orf_next_stops[0]+3])     # Should be a stop codon
    
    if len(orf_lens)==0:
        return None, None
    else:
        maxlen = max(orf_lens)
        return maxlen, orf_start_pos[orf_lens.index(maxlen)]
        # Note does not deal with ties
    
maxlens = {}
for key,value in seq_dict.items():
    #max_len_start = [orf(value.seq,start) for start in range(3)]
    max_len_start = [orf(value.seq,start) for start in [2]]
    # List of tuples
    print('Max lengths per frame: ', max_len_start)
    lens_not_none = [i for i in max_len_start if i != (None, None)]
    if len(lens_not_none) > 0:
        maxlens[key] = max(lens_not_none)

# Test accessing by identifier
#maxlens['gi|142022655|gb|EQ086233.1|43']
#maxlens['gi|142022655|gb|EQ086233.1|41']
len_id = maxlens['gi|142022655|gb|EQ086233.1|16']
print('Max len ORF of gi|142022655|gb|EQ086233.1|16:', len_id)

# Unzip tuples
res = list(zip(*maxlens.values()))
maxlens = res[0]
maxlen_pos = res[1]

maxlens
maxlen_pos

max_of_all = max(maxlens)
max_of_all_pos = maxlen_pos[maxlens.index(max_of_all)] 
max_of_all_id = identifier[maxlens.index(max_of_all)]

print('Sequence with longest ORF: ', max_of_all_id)
print('Starting position of longest ORF: ', max_of_all_pos)
print('Nucleotide length of longest ORF: ', max_of_all)


Max lengths per frame:  [(588, 2856)]
Max lengths per frame:  [(147, 621)]
Max lengths per frame:  [(285, 1641)]
Max lengths per frame:  [(324, 699)]
Max lengths per frame:  [(225, 429)]
Max lengths per frame:  [(249, 1374)]
Max lengths per frame:  [(189, 90)]
Max lengths per frame:  [(None, None)]
Max lengths per frame:  [(213, 66)]
Max lengths per frame:  [(711, 2334)]
Max lengths per frame:  [(204, 57)]
Max lengths per frame:  [(1401, 3096)]
Max lengths per frame:  [(1644, 1440)]
Max lengths per frame:  [(132, 348)]
Max lengths per frame:  [(219, 1278)]
Max lengths per frame:  [(171, 759)]
Max lengths per frame:  [(None, None)]
Max lengths per frame:  [(1821, 636)]
Max len ORF of gi|142022655|gb|EQ086233.1|16: (1644, 1440)
Sequence with longest ORF:  gi|142022655|gb|EQ086233.1|277
Starting position of longest ORF:  636
Nucleotide length of longest ORF:  1821


In [29]:
print(maxlens)

(213, 363, 918, 594, 1608, 1686, 507, 159, 552, 1311, 639, 720, 291, 1218, 1107, 537, 477, 564, 219, 741, 678, 636, 153, 534, 639)


In [30]:
print(maxlen_pos)

(367, 107, 1194, 1772, 141, 2824, 825, 126, 86, 125, 216, 1347, 1427, 150, 3592, 219, 987, 410, 794, 930, 365, 606, 413, 2, 1633)


### Part 3
We will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.

In [126]:
# Oh, not tandem repeats, just repeats

# chop up into frames of length n, tally up
from collections import Counter

def find_repeats(n):
    seg_list = []
    for key,value in seq_dict.items():
        seq = value.seq
        for i in range(len(seq)-n):
            seg = seq[i:i+n]
            seg_list.append(seg)
    
    counts = Counter(seg_list)
    
    vals = list(counts.values())
    key_list = list(counts.keys())
    maxcount = max(vals)
    maxind =  vals.index(maxcount)
    maxseg = key_list[maxind]
    print(f'Manual: {maxseg} : {maxcount} \n')  
    
    # Actually Counter has built in
    print(counts.most_common(1))
    
    print(counts)
   

        
        # need a count3
    # Read in n sequences, shift by 1 up to n and compare
    # 
    
    # repeated n times or length of repeat is n?
    # what about ACACAACA?

In [132]:
find_repeats(7)

Manual: CGCGCCG : 63 

[(Seq('CGCGCCG'), 63)]
Counter({Seq('CGCGCCG'): 63, Seq('CGCCGCG'): 62, Seq('GCCGCGC'): 61, Seq('GCGCGCG'): 59, Seq('GCGCGGC'): 58, Seq('CGCGGCG'): 57, Seq('CGCGACG'): 57, Seq('CGCGCGC'): 56, Seq('GCCGCCG'): 52, Seq('CGGCGCG'): 51, Seq('GCGGCCG'): 50, Seq('GCGCCGC'): 47, Seq('CGCGCGG'): 47, Seq('CGGCCGC'): 46, Seq('CGCGGCC'): 45, Seq('GCGCGCC'): 45, Seq('GGCCGCG'): 45, Seq('GCCGGCG'): 44, Seq('GATCGCG'): 43, Seq('GGCGCGC'): 42, Seq('GCGCCGA'): 42, Seq('CGGCGGC'): 41, Seq('CGCCGGC'): 41, Seq('GCTGCCG'): 41, Seq('CGACGGC'): 40, Seq('CCGGCCG'): 40, Seq('CGACGCG'): 40, Seq('GCCGACG'): 40, Seq('GCCGGCC'): 39, Seq('CGCGAGC'): 39, Seq('CGCGCTG'): 38, Seq('GCCGAGC'): 38, Seq('CGTCGCG'): 38, Seq('CCGGCGC'): 38, Seq('GCTCGCG'): 37, Seq('GCGACGC'): 37, Seq('CGAGCGC'): 37, Seq('GCCGCGA'): 37, Seq('CGATCGC'): 37, Seq('GCGATCG'): 36, Seq('TGCGCGC'): 36, Seq('GCGCTGC'): 36, Seq('GGCCGGC'): 35, Seq('TCGGCGA'): 35, Seq('CGCGATC'): 35, Seq('CTGCGCG'): 35, Seq('CGGCCGG'): 34, Seq('

In [120]:
counts

Counter({Seq('GCGCGA'): 14, Seq('GCGAGC'): 12, Seq('GCGACG'): 11, Seq('CGAGCG'): 11, Seq('CGAGCA'): 10, Seq('GCGGCC'): 10, Seq('CCGAGC'): 9, Seq('CGCGAG'): 9, Seq('CGCCGA'): 9, Seq('CGGCGC'): 9, Seq('CGACGA'): 8, Seq('GAGCAG'): 8, Seq('AGCAGC'): 8, Seq('GCACGC'): 8, Seq('CGCGGC'): 8, Seq('GCCGAG'): 8, Seq('AGCGCG'): 7, Seq('CGCGCG'): 7, Seq('ACGCCG'): 7, Seq('AACGCG'): 7, Seq('CGAGCC'): 7, Seq('GGCGCG'): 7, Seq('CGTCGA'): 7, Seq('CCGCGC'): 6, Seq('CAGCGC'): 6, Seq('GCGCGC'): 6, Seq('CGATCG'): 6, Seq('GGCGAC'): 6, Seq('CGGCCG'): 6, Seq('CGCGCC'): 6, Seq('ACGGCG'): 6, Seq('CGACGC'): 6, Seq('ACGCGG'): 6, Seq('CGCACG'): 6, Seq('CGCGAT'): 6, Seq('GCGGCG'): 6, Seq('TCGCGA'): 6, Seq('TTCGCG'): 6, Seq('TCGCGC'): 6, Seq('GAGCGG'): 6, Seq('CATCGC'): 6, Seq('GCGCCG'): 6, Seq('GTCGAT'): 6, Seq('GCATCG'): 6, Seq('CGCGTC'): 6, Seq('CGCTCG'): 5, Seq('TCGATG'): 5, Seq('CGGCGA'): 5, Seq('TGACGA'): 5, Seq('AGCCCG'): 5, Seq('CGCTTC'): 5, Seq('GCCGGC'): 5, Seq('GTGAGC'): 5, Seq('GCAGCA'): 5, Seq('CGTCGC')

In [111]:
n = 6
for key,value in seq_dict.items():
    seq = value.seq
    counts = find_repeats(seq,n)
    vals = list(counts.values())
    key_list = list(counts.keys())
    maxcount = max(counts.values())
    maxind =  vals.index(maxcount)
    maxseg = key_list[maxind]
    print(f'{maxseg} : {maxcount} \n')
    
    # Actually Counter has built in
    counts.most_common(1)


GCGCTG : 4 

GCGCGC : 4 

GCTCGA : 8 

CCGGCG : 7 

CGCCGC : 17 

CGCCGC : 27 

CGGCGC : 10 

CCGGGT : 5 

CGGGCG : 4 

CGGCGC : 12 

GCGCGA : 10 

CGCCGG : 11 

GCCGGC : 15 

CGGCGC : 7 

CGCGCG : 27 

CGGCGA : 9 

CGCGCC : 17 

TGCGCG : 12 

CGCGCG : 8 

GCGGCG : 11 

TCGACC : 6 

GGCGGC : 8 

GCCGGC : 12 

CGCGGC : 5 

GCGCGA : 14 



In [66]:
counts

Counter({Seq('CGC'): 495, Seq('GCG'): 429, Seq('TCG'): 407, Seq('CGA'): 330, Seq('GCC'): 308, Seq('GGC'): 286, Seq('CGG'): 275, Seq('AGC'): 253, Seq('GCT'): 253, Seq('ACG'): 242, Seq('CCG'): 242, Seq('GCA'): 231, Seq('CGT'): 220, Seq('TGG'): 220, Seq('TTC'): 220, Seq('CTG'): 209, Seq('ATC'): 209, Seq('GGG'): 198, Seq('GAA'): 198, Seq('GTC'): 198, Seq('GAT'): 198, Seq('CCA'): 187, Seq('TGC'): 187, Seq('CAG'): 176, Seq('CAC'): 176, Seq('CTT'): 176, Seq('GGA'): 176, Seq('GAC'): 176, Seq('GGT'): 176, Seq('CTC'): 165, Seq('TTG'): 165, Seq('ATG'): 165, Seq('TGA'): 165, Seq('AAG'): 154, Seq('CCT'): 154, Seq('CAT'): 154, Seq('GTT'): 154, Seq('GAG'): 154, Seq('AGG'): 143, Seq('CAA'): 143, Seq('TCC'): 143, Seq('ACC'): 132, Seq('TCT'): 132, Seq('TCA'): 121, Seq('GTA'): 121, Seq('AAT'): 121, Seq('AGT'): 110, Seq('GTG'): 110, Seq('AAA'): 110, Seq('ACA'): 110, Seq('AAC'): 99, Seq('ATA'): 88, Seq('CCC'): 88, Seq('TAG'): 77, Seq('TAC'): 77, Seq('ATT'): 77, Seq('TGT'): 77, Seq('TAT'): 66, Seq('AGA'): 5

[(Seq('CGC'), 45)]

In [91]:
len(seq_dict['gi|142022655|gb|EQ086233.1|43'].seq)

990